# OR-Tools Cheatsheet

OR-Tools is open source software for combinatorial optimization.

It includes solvers for:
- Constraint Programming
- Linear and Mixed-Integer Programming
- Vehicle Routing
- Graph Algorithms

[Home](https://developers.google.com/optimization)

## API Naming Convention

- `new_<data-type>_var`: Declare a variable of `data-type`
    - ex: `new_int_var`
- `add_<constraint_type>`: Declare a constraint for the programming problem
    - ex: `add_abs_equality`
- `maximize` and `minimize`: Declare a maximizing/minimizing objective for the programming problem

In [1]:
# Example
import ortools.sat.python.cp_model as cp_model

In [2]:
# Declare a constraint programming problem
model = cp_model.CpModel()
var_x = model.new_int_var(1, 5, "var_x") # x is a variable in [1, 5]
var_y = model.new_int_var(3, 10, "var_y") # y is a variable in [3, 10]
objective = var_x - var_y
model.add(var_x >= var_y) # x must be larger or equal to y
model.maximize(objective)

In [3]:
# Solve the constraint programming problem
solver = cp_model.CpSolver()
status = solver.solve(model)

- Possible values for the `status`
  - `OPTIMAL`
  - `FEASIBLE`
  - `INFEASIBLE`
  - `MODEL_INVALID`
  - `UNKNOWN`

The meaning of these status can be found [here](https://developers.google.com/optimization/cp/cp_solver#cp-sat_return_values).

In [4]:
if status == cp_model.OPTIMAL:
    objective_value = solver.value(objective)
    x = solver.value(var_x)
    y = solver.value(var_y)
    print(f"the optimal x and y are: {(x, y)}")
    print(f"the maximum of `x - y` is {objective_value}")
else:
    print("Fail to solve the cp problem, so sad :(")

the optimal x and y are: (5, 3)
the maximum of `x - y` is 2


## Some Useful API You may Need for Solving Mem Allocation

- `model.new_int_var`
    - Since the memory offset we want to solve are integers.
- `model.new_interval_var`
    - declare intervals to represent the memory blocks of the tensors.
    - the `start` and `end` of the interval could be integer variables.
- `model.add_no_overlap`
    - declare no-overlapping constraint on intervals

## Allocation Plan Data

In [5]:
import pickle

In [6]:
with open("./labs/simple_model_mem_alloc_data.pkl", "rb") as fid:
    mem_alloc_data = pickle.load(fid)
mem_alloc_data.keys()

dict_keys(['tensors_meta', 'op_inputs_map', 'op_outputs_map', 'ops_topo_order'])

- `tensors_meta`(`dict[str, tuple]`): a dictionary which maps tensor names to its meta data
    - the meta data of the tensor is a tuple: `(shape, dtype, itemsize)`
- `op_inputs_map`(`dict[str, list]`): a dictionary which maps op/node names to its input tensors
    - the value are list of names of input tensors of the node
- `op_outputs_map`(`dict[str, list]`): similar to `op_inputs_map` where the values are the names of the output tensors of the node
- `ops_topo_order` (`list[str]`): the names of the op/nodes in the graph in topological ordering